In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import numpy as np
from sklearn.cluster import *

/usr/local/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

/usr/local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:35: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _k_means
/usr/local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._k_means_elkan import k_means_elkan
/usr/local/lib/python2.7/site-packages/sklearn/cluster/hierarchical.py:23: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _hierarchical
/usr/local/lib/python2.7/site-packages/sklearn/cluster/dbscan_.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._dbscan_inner import dbscan_inner


In [17]:
def _parse_blob(blob, wo_benchmark=True):
    lines  = blob.replace(',', '.').split("\n")[1:-1]
    fundos = [ l.strip()[:30] for l in lines[::2] ]
    corr   = [ l.split("\t") for l in lines[1::2] ]

    for i,v in enumerate(corr):
        for j,_ in enumerate(v[i:]):
            corr[i][i+j] = corr[j+i][i] 

    if wo_benchmark:
        corr   = [ c[1:] for c in corr[1:] ]
        fundos = fundos[1:]

    return fundos, np.array(corr).astype('float')

def _print(cl, fundos, label):
    print  label + " ------\n" + "\n".join(map(str,sorted(zip(cl.labels_, fundos), key=lambda x: x[0])))

def _to_df(*blobs):
    dfs = []
    for b in blobs:
        fundos, corr = _parse_blob(b, wo_benchmark=True)
        dfs.append( pd.DataFrame(corr, index=fundos) )
    return pd.concat(dfs, join='inner', axis=1)

def process(blob, wo_benchmark=True, k=5):
    fundos, corr = _parse_blob(blob, wo_benchmark=wo_benchmark)
    _cluster(fundos, corr, k=k)

def _cluster(fundos, corr, k=5):
    cl = AffinityPropagation(max_iter=500, convergence_iter=100)
    cl.fit(corr)
    _print(cl, fundos, "affinity")

    cl = AgglomerativeClustering(affinity='precomputed', n_clusters=k, linkage='complete')
    cl.fit(-corr)
    _print(cl, fundos, "agg precomputed")
    
    cl = AgglomerativeClustering(n_clusters=k)
    cl.fit(-corr)
    _print(cl, fundos, "agg")

#     return corr

## Ações

In [3]:
acoes_baixa = """
 IBOVESPA
1,00									
 OCEANA LONG BIASED FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO DE AÇÕES
0,90	1,00								
 HIX CAPITAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES
0,66	0,72	1,00							
 ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I
0,46	0,47	0,40	1,00						
 IBIUNA EQUITIES FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,91	0,88	0,69	0,46	1,00					
 XP LONG BIASED FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,81	0,81	0,63	0,40	0,80	1,00				
 SPX FALCON FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO DE AÇÕES
0,74	0,77	0,59	0,40	0,77	0,73	1,00			
 BRASIL CAPITAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,68	0,74	0,72	0,39	0,68	0,63	0,61	1,00		
 BOGARI VALUE FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO DE AÇÕES
0,90	0,91	0,74	0,46	0,90	0,81	0,78	0,73	1,00	
 ATMOS AÇÕES FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,83	0,83	0,72	0,42	0,85	0,78	0,75	0,65	0,88	1,00
"""

In [4]:
acoes_alta = """
 IBOVESPA
1,00									
 OCEANA LONG BIASED FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO DE AÇÕES
0,88	1,00								
 HIX CAPITAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES
0,75	0,81	1,00							
 ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I
0,66	0,65	0,65	1,00						
 IBIUNA EQUITIES FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,93	0,90	0,76	0,66	1,00					
 XP LONG BIASED FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,86	0,85	0,72	0,64	0,86	1,00				
 SPX FALCON FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO DE AÇÕES
0,80	0,81	0,74	0,72	0,83	0,81	1,00			
 BRASIL CAPITAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,85	0,89	0,82	0,68	0,88	0,81	0,83	1,00		
 BOGARI VALUE FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO DE AÇÕES
0,90	0,91	0,81	0,70	0,92	0,86	0,87	0,90	1,00	
 ATMOS AÇÕES FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,80	0,87	0,75	0,65	0,85	0,78	0,78	0,84	0,87	1,00
"""

In [5]:
acoes = """
 IBOVESPA
1,00									
 OCEANA LONG BIASED FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO DE AÇÕES
0,89	1,00								
 HIX CAPITAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO EM AÇÕES
0,71	0,75	1,00							
 ALASKA BLACK FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES - BDR NÍVEL I
0,57	0,55	0,55	1,00						
 IBIUNA EQUITIES FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,92	0,87	0,73	0,59	1,00					
 XP LONG BIASED FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,84	0,83	0,68	0,54	0,83	1,00				
 SPX FALCON FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO DE AÇÕES
0,77	0,78	0,67	0,60	0,80	0,77	1,00			
 BRASIL CAPITAL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,77	0,81	0,77	0,57	0,79	0,72	0,73	1,00		
 BOGARI VALUE FUNDO DE INVESTIMENTO EM COTAS DE FUNDO DE INVESTIMENTO DE AÇÕES
0,89	0,88	0,78	0,62	0,91	0,83	0,83	0,83	1,00	
 ATMOS AÇÕES FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO EM AÇÕES
0,80	0,83	0,73	0,58	0,85	0,77	0,76	0,76	0,87	1,00
"""


In [129]:
process(acoes, wo_benchmark=False)

(0, ' ALASKA BLACK FUNDO DE INVESTI')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' XP LONG BIASED FUNDO DE INVES')
(1, ' BOGARI VALUE FUNDO DE INVESTI')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(2, ' SPX FALCON FUNDO DE INVESTIME')
(3, ' HIX CAPITAL FUNDO DE INVESTIM')
(3, ' BRASIL CAPITAL FUNDO DE INVES')
------
(0, ' HIX CAPITAL FUNDO DE INVESTIM')
(0, ' BRASIL CAPITAL FUNDO DE INVES')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(2, ' IBOVESPA')
(2, ' OCEANA LONG BIASED FUNDO DE I')
(2, ' IBIUNA EQUITIES FUNDO DE INVE')
(2, ' XP LONG BIASED FUNDO DE INVES')
(2, ' BOGARI VALUE FUNDO DE INVESTI')
(3, ' ALASKA BLACK FUNDO DE INVESTI')
(4, ' SPX FALCON FUNDO DE INVESTIME')
------
(0, ' XP LONG BIASED FUNDO DE INVES')
(0, ' SPX FALCON FUNDO DE INVESTIME')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' BOGARI VALUE FUNDO DE INVESTI')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO

In [122]:
process(acoes_baixa, wo_benchmark=False)

(0, ' ALASKA BLACK FUNDO DE INVESTI')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' XP LONG BIASED FUNDO DE INVES')
(1, ' SPX FALCON FUNDO DE INVESTIME')
(1, ' BOGARI VALUE FUNDO DE INVESTI')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(2, ' HIX CAPITAL FUNDO DE INVESTIM')
(2, ' BRASIL CAPITAL FUNDO DE INVES')
------
(0, ' IBOVESPA')
(0, ' OCEANA LONG BIASED FUNDO DE I')
(0, ' IBIUNA EQUITIES FUNDO DE INVE')
(0, ' XP LONG BIASED FUNDO DE INVES')
(0, ' BOGARI VALUE FUNDO DE INVESTI')
(0, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(1, ' SPX FALCON FUNDO DE INVESTIME')
(2, ' BRASIL CAPITAL FUNDO DE INVES')
(3, ' HIX CAPITAL FUNDO DE INVESTIM')
(4, ' ALASKA BLACK FUNDO DE INVESTI')
------
(0, ' XP LONG BIASED FUNDO DE INVES')
(0, ' SPX FALCON FUNDO DE INVESTIME')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' BOGARI VALUE FUNDO DE INVESTI')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO

In [125]:
process(acoes_alta, wo_benchmark=False)

(0, ' HIX CAPITAL FUNDO DE INVESTIM')
(1, ' ALASKA BLACK FUNDO DE INVESTI')
(2, ' IBOVESPA')
(2, ' OCEANA LONG BIASED FUNDO DE I')
(2, ' IBIUNA EQUITIES FUNDO DE INVE')
(2, ' XP LONG BIASED FUNDO DE INVES')
(2, ' BRASIL CAPITAL FUNDO DE INVES')
(2, ' BOGARI VALUE FUNDO DE INVESTI')
(2, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(3, ' SPX FALCON FUNDO DE INVESTIME')
------
(0, ' OCEANA LONG BIASED FUNDO DE I')
(0, ' BRASIL CAPITAL FUNDO DE INVES')
(0, ' BOGARI VALUE FUNDO DE INVESTI')
(0, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(1, ' IBOVESPA')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' XP LONG BIASED FUNDO DE INVES')
(2, ' HIX CAPITAL FUNDO DE INVESTIM')
(3, ' ALASKA BLACK FUNDO DE INVESTI')
(4, ' SPX FALCON FUNDO DE INVESTIME')
------
(0, ' XP LONG BIASED FUNDO DE INVES')
(0, ' SPX FALCON FUNDO DE INVESTIME')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' BRASIL CAPITAL FUNDO DE INVES')
(1, ' BOGARI VALUE FUNDO DE INVESTI')

In [182]:
df = _to_df(acoes_baixa, acoes_alta)
_cluster(df.index, df.values, k=5)

(0, ' ALASKA BLACK FUNDO DE INVESTI')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' XP LONG BIASED FUNDO DE INVES')
(1, ' SPX FALCON FUNDO DE INVESTIME')
(1, ' BOGARI VALUE FUNDO DE INVESTI')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(2, ' HIX CAPITAL FUNDO DE INVESTIM')
(2, ' BRASIL CAPITAL FUNDO DE INVES')
------
(0, ' IBOVESPA')
(0, ' OCEANA LONG BIASED FUNDO DE I')
(0, ' IBIUNA EQUITIES FUNDO DE INVE')
(0, ' XP LONG BIASED FUNDO DE INVES')
(0, ' BOGARI VALUE FUNDO DE INVESTI')
(0, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO DE INVEST')
(1, ' SPX FALCON FUNDO DE INVESTIME')
(2, ' BRASIL CAPITAL FUNDO DE INVES')
(3, ' HIX CAPITAL FUNDO DE INVESTIM')
(4, ' ALASKA BLACK FUNDO DE INVESTI')
------
(0, ' XP LONG BIASED FUNDO DE INVES')
(0, ' SPX FALCON FUNDO DE INVESTIME')
(1, ' IBOVESPA')
(1, ' OCEANA LONG BIASED FUNDO DE I')
(1, ' IBIUNA EQUITIES FUNDO DE INVE')
(1, ' BOGARI VALUE FUNDO DE INVESTI')
(1, ' ATMOS A\xc3\x87\xc3\x95ES FUNDO

## Multi

In [ ]:
#selic
Nov/2016 -> queda
Jul/2015 -> manutencao
Jan/2013 -> subida

#dolar
Nov/2014 -> subida
Nov/2015 -> pico
Jan/2016 -> queda
Jan/2017 -> manutencao
Jan/2018 -> subida

In [6]:
# de 12/2013 a Jul/2015
multi_2014 = """
     CDI
    1,00					
     SPX NIMITZ FEEDER FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
    0,06	1,00				
     KAPITALO KAPPA FIN FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
    -0,01	0,01	1,00			
     GARDE D'ARTAGNAN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
    0,12	0,55	0,17	1,00		
     BAHIA AM MARAÚ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
    0,03	0,37	0,26	0,28	1,00	
     ABSOLUTE HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
    0,04	0,27	0,22	0,37	0,22	1,00
"""

In [7]:
# de Jul/2015 ate Nov/2016
multi_2016 = """
 CDI
1,00							
 SPX NIMITZ FEEDER FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
-0,10	1,00						
 KAPITALO KAPPA FIN FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
-0,07	0,27	1,00					
 GARDE D'ARTAGNAN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,01	0,15	0,44	1,00				
 KINEA CHRONOS FUNDO DE INVESTIMENTO MULTIMERCADO
-0,05	-0,08	0,17	0,18	1,00			
 BAHIA AM MARAÚ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
-0,00	0,53	0,30	0,31	0,10	1,00		
 ABSOLUTE HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
-0,04	0,41	0,26	0,20	0,08	0,57	1,00	
 VINTAGE MACRO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
-0,07	0,00	0,22	0,15	0,53	0,13	-0,01	1,00
"""

In [8]:
multi_2017 = """"
 CDI
1,00							
 SPX NIMITZ FEEDER FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,05	1,00						
 KAPITALO KAPPA FIN FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,02	0,42	1,00					
 GARDE D'ARTAGNAN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,10	0,47	0,81	1,00				
 KINEA CHRONOS FUNDO DE INVESTIMENTO MULTIMERCADO
0,06	0,35	0,70	0,70	1,00			
 BAHIA AM MARAÚ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,04	0,56	0,80	0,86	0,77	1,00		
 ABSOLUTE HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
-0,00	0,36	0,81	0,73	0,63	0,74	1,00	
 VINTAGE MACRO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,07	0,39	0,78	0,76	0,70	0,72	0,68	1,00
"""

In [9]:
# depois de mai/2017
multi_todos = """
SPX NIMITZ FEEDER FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,04	1,00							
 ADAM MACRO STRATEGY II FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,08	0,22	1,00						
 KAPITALO KAPPA FIN FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,06	0,41	0,54	1,00					
 GARDE D'ARTAGNAN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,14	0,38	0,49	0,70	1,00				
 KINEA CHRONOS FUNDO DE INVESTIMENTO MULTIMERCADO
0,14	0,12	0,32	0,46	0,44	1,00			
 BAHIA AM MARAÚ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,18	0,46	0,38	0,71	0,69	0,54	1,00		
 ABSOLUTE HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,08	0,31	0,33	0,54	0,50	0,22	0,58	1,00	
 VINTAGE MACRO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,07	0,32	0,42	0,63	0,62	0,56	0,60	0,31	1,00
"""


In [10]:
multi = """
 CDI
1,00								
 SPX NIMITZ FEEDER FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,06	1,00							
 KAPITALO KAPPA FIN FUNDO DE INVESTIMENTO EM QUOTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,03	0,42	1,00						
 GARDE D'ARTAGNAN FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,09	0,47	0,80	1,00					
 KINEA CHRONOS FUNDO DE INVESTIMENTO MULTIMERCADO
0,05	0,35	0,70	0,70	1,00				
 BAHIA AM MARAÚ FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,04	0,55	0,78	0,84	0,75	1,00			
 ADAM MACRO II FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,06	0,32	0,69	0,73	0,58	0,68	1,00		
 ABSOLUTE HEDGE FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,01	0,36	0,80	0,73	0,63	0,75	0,63	1,00	
 VINTAGE MACRO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO
0,07	0,39	0,77	0,75	0,69	0,70	0,62	0,67	1,00
"""

In [195]:
process(multi, k=5)

affinity ------
(0, ' SPX NIMITZ FEEDER FUNDO DE IN')
(1, ' KAPITALO KAPPA FIN FUNDO DE I')
(1, " GARDE D'ARTAGNAN FUNDO DE INV")
(1, ' BAHIA AM MARA\xc3\x9a FUNDO DE INVE')
(1, ' ABSOLUTE HEDGE FUNDO DE INVES')
(1, ' VINTAGE MACRO FUNDO DE INVEST')
(2, ' KINEA CHRONOS FUNDO DE INVEST')
(3, ' ADAM MACRO II FUNDO DE INVEST')
agg precomputed ------
(0, ' KAPITALO KAPPA FIN FUNDO DE I')
(0, " GARDE D'ARTAGNAN FUNDO DE INV")
(0, ' BAHIA AM MARA\xc3\x9a FUNDO DE INVE')
(0, ' ABSOLUTE HEDGE FUNDO DE INVES')
(1, ' ADAM MACRO II FUNDO DE INVEST')
(2, ' VINTAGE MACRO FUNDO DE INVEST')
(3, ' SPX NIMITZ FEEDER FUNDO DE IN')
(4, ' KINEA CHRONOS FUNDO DE INVEST')
agg ------
(0, ' KAPITALO KAPPA FIN FUNDO DE I')
(0, ' ABSOLUTE HEDGE FUNDO DE INVES')
(0, ' VINTAGE MACRO FUNDO DE INVEST')
(1, " GARDE D'ARTAGNAN FUNDO DE INV")
(1, ' BAHIA AM MARA\xc3\x9a FUNDO DE INVE')
(2, ' ADAM MACRO II FUNDO DE INVEST')
(3, ' SPX NIMITZ FEEDER FUNDO DE IN')
(4, ' KINEA CHRONOS FUNDO DE INVEST')


In [19]:
process(multi_2014, k=3)

affinity ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(1, 'SPX NIMITZ FEEDER FUNDO DE INV')
(1, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'ABSOLUTE HEDGE FUNDO DE INVEST')
agg precomputed ------
(0, 'SPX NIMITZ FEEDER FUNDO DE INV')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(0, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(2, 'KAPITALO KAPPA FIN FUNDO DE IN')
agg ------
(0, 'SPX NIMITZ FEEDER FUNDO DE INV')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(0, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'KAPITALO KAPPA FIN FUNDO DE IN')
(2, 'ABSOLUTE HEDGE FUNDO DE INVEST')


In [21]:
process(multi_2016, k=4)

affinity ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'SPX NIMITZ FEEDER FUNDO DE INV')
(1, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(2, 'KINEA CHRONOS FUNDO DE INVESTI')
(2, 'VINTAGE MACRO FUNDO DE INVESTI')
agg precomputed ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'KINEA CHRONOS FUNDO DE INVESTI')
(1, 'VINTAGE MACRO FUNDO DE INVESTI')
(2, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(2, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(3, 'SPX NIMITZ FEEDER FUNDO DE INV')
agg ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'KINEA CHRONOS FUNDO DE INVESTI')
(1, 'VINTAGE MACRO FUNDO DE INVESTI')
(2, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(2, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(3, 'SPX NIMITZ FEEDER FUNDO DE INV')


In [22]:
process(multi_2017, k=4)

affinity ------
(0, 'SPX NIMITZ FEEDER FUNDO DE INV')
(1, 'KAPITALO KAPPA FIN FUNDO DE IN')
(1, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(1, 'VINTAGE MACRO FUNDO DE INVESTI')
(2, 'KINEA CHRONOS FUNDO DE INVESTI')
agg precomputed ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(0, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(0, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(1, 'KINEA CHRONOS FUNDO DE INVESTI')
(2, 'VINTAGE MACRO FUNDO DE INVESTI')
(3, 'SPX NIMITZ FEEDER FUNDO DE INV')
agg ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(0, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(0, 'VINTAGE MACRO FUNDO DE INVESTI')
(1, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(2, 'KINEA CHRONOS FUNDO DE INVESTI')
(3, 'SPX NIMITZ FEEDER FUNDO DE INV')


In [23]:
df = _to_df(multi_2014, multi_2016, multi_2017)
_cluster(df.index, df.values, k=3)

affinity ------
(0, 'SPX NIMITZ FEEDER FUNDO DE INV')
(1, 'KAPITALO KAPPA FIN FUNDO DE IN')
(1, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'ABSOLUTE HEDGE FUNDO DE INVEST')
agg precomputed ------
(0, 'SPX NIMITZ FEEDER FUNDO DE INV')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(0, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(1, 'ABSOLUTE HEDGE FUNDO DE INVEST')
(2, 'KAPITALO KAPPA FIN FUNDO DE IN')
agg ------
(0, 'KAPITALO KAPPA FIN FUNDO DE IN')
(0, "GARDE D'ARTAGNAN FUNDO DE INVE")
(1, 'SPX NIMITZ FEEDER FUNDO DE INV')
(2, 'BAHIA AM MARA\xc3\x9a FUNDO DE INVES')
(2, 'ABSOLUTE HEDGE FUNDO DE INVEST')
